In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib widget

import torch
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import wandb

import neural_network_lyapunov.examples.quadrotor3d.quadrotor as quadrotor
import neural_network_lyapunov.utils as utils
import neural_network_lyapunov.feedback_system as feedback_system
import neural_network_lyapunov.lyapunov as lyapunov
import neural_network_lyapunov.train_lyapunov as train_lyapunov

In [ ]:
wandb.init(project="px4-lyapunov")

In [ ]:
%run -d train_quadrotor_px4_demo.py data/px4_data.pt \
    --train_forward_model \
    --save_forward_model models/forward_model.pt \
    --forward_model_only

In [ ]:
%run -d train_quadrotor_px4_demo.py data/px4_data.pt \
    --train_forward_model \
    --load_forward_model models/forward_model.pt \
    --save_forward_model models/forward_model.pt \
    --forward_model_only

In [ ]:
%run -d train_quadrotor_px4_demo.py data/px4_data.pt \
    --load_forward_model models/forward_model.pt \
    --train_on_samples

In [ ]:
%run -d train_quadrotor_px4_demo.py data/px4_data.pt \
    --load_forward_model models/forward_model.pt \
    --load_folder models

## Checking model

In [ ]:
dynamics_relu = torch.load('models/forward_model.pt')
hover_thrust = .705
xu_equ = torch.tensor([0., 0., 0., 0., 0., 0., hover_thrust], dtype=torch.float64)

In [ ]:
data, labels = torch.load('data/px4_data.pt')

val_ratio = .1
num_val = int(val_ratio * data.shape[0])

val_samples = np.random.choice(data.shape[0], num_val, replace=False)
train_samples = np.array([i for i in range(data.shape[0]) if i not in val_samples])

data_train = data[train_samples, :]
labels_train = labels[train_samples, :]
data_val = data[val_samples, :]
labels_val = labels[val_samples, :]

In [ ]:
y_pred_val = (dynamics_relu.eval()(data_val) - dynamics_relu.eval()(xu_equ)).detach()
print(torch.mean(torch.square(y_pred_val - labels_val)))

In [ ]:
for i in range(6):
    plt.figure()
    plt.plot(labels_val[:, i], '.-')
    plt.plot(y_pred_val[:, i], '.-')
    plt.legend(['true', 'predicted'])
    plt.show()

In [ ]:
n_start = 0
n_end = n_start + 1000
data_check = data_train[n_start:n_end, :]
labels_check = labels_train[n_start:n_end, :]
y_pred_train = (dynamics_relu.eval()(data_check) - dynamics_relu.eval()(xu_equ)).detach()
print(torch.mean(torch.square(y_pred_train - labels_check)))

In [ ]:
for i in range(6):
    plt.figure()
    plt.plot(labels_check[:, i])
    plt.plot(y_pred_train[:, i])
    plt.legend(['true', 'predicted'])
    plt.show()